In [1]:
%pip install --upgrade git+https://github.com/keras-team/keras-cv -q --break-system-packages
# %pip install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 --break-system-packages

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
import cv2
import random

import numpy as np

/home/gabriela/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-20 18:11:48.751684: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 18:11:49.026384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737407509.127237    1462 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737407509.161428    1462 cuda_blas.cc:1418] Unable t

In [3]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0
IMAGE_WIDTH = 3000
IMAGE_HEIGHT = 4000

TF_ENABLE_ONEDNN_OPTS=0
TF_DISABLE_MKL=1

In [4]:
class_ids = [
    "neutrofilo",
    "linfocito",
    "monocito",
    "bastonete",
    "metamielocito",
    "eosinofilo",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
path_images = "/home/gabriela/projetos/yolov8keras/Bach1"
path_annot = "/home/gabriela/projetos/yolov8keras/Bach1/annotations"

path_test_img = "/home/gabriela/projetos/yolov8keras/test"
path_test_annot = "/home/gabriela/projetos/yolov8keras/test/annotations"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

xml_test_files = sorted(
    [
        os.path.join(path_test_annot, file_name)
        for file_name in os.listdir(path_test_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)

jpg_test_files = sorted(
    [
        os.path.join(path_test_img, file_name)
        for file_name in os.listdir(path_test_img)
        if file_name.endswith(".jpg")
    ]
)

In [5]:
class_mapping[0]

'neutrofilo'

In [6]:
#read the objects in the annotation xml file
def parse_annotation(xml_file, path_img):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_img, image_name)

    boxes = []
    classes = []
    for obj in root.find("objects"):
        cls = obj.tag
        # print(cls)
        classes.append(cls)

        if(obj.find("bbox") != None):
            bbox = obj.find("bbox")
            min_x, min_y, width, height = normalize_bounding_box(float(bbox.find("x").text), float(bbox.find("y").text), float(bbox.find("width").text), float(bbox.find("height").text))
            boxes.append([min_x, min_y, width, height])
        else:
            xCoords = []
            yCoords = []
            for coord in obj:
                if (coord.tag.find("x") != -1):
                    xCoords.append(float(coord.text))
                elif (coord.tag.find("y") != -1):
                    yCoords.append(float(coord.text))

            xmin, ymin, width, height = calculate_bounding_box_normalized(xCoords, yCoords)
            boxes.append([xmin, ymin, width, height])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids

#calculate bounding boxes with points provided
def calculate_bounding_box_normalized(xCoords, yCoords):
    points = np.array([xCoords, yCoords])

    min_x = float(np.min(points[0, :]))
    min_x = (min_x/IMAGE_WIDTH)*640
    min_y = float(np.min(points[1, :]))
    # min_y = ((IMAGE_HEIGHT - min_y)/IMAGE_HEIGHT)*640
    min_y = (min_y/IMAGE_HEIGHT)*640

    max_x = float(np.max(points[0, :]))
    max_x = (max_x/IMAGE_WIDTH)*640
    max_y = float(np.max(points[1, :]))
    # max_y = ((IMAGE_HEIGHT - max_y)/IMAGE_HEIGHT)*640
    max_y = (max_y/IMAGE_HEIGHT)*640

    width =  max_x - min_x
    height = max_y - min_y

    return (min_x, min_y, width, height)

def normalize_bounding_box(xmin, ymin, width, height):

    min_x = (xmin/IMAGE_WIDTH)*640
    # min_y = ((IMAGE_HEIGHT - ymin)/IMAGE_HEIGHT)*640
    min_y = (ymin/IMAGE_HEIGHT)*640
    width = (width/IMAGE_WIDTH)*640
    height = (height/IMAGE_HEIGHT)*640

    return (min_x, min_y, width, height)

In [7]:
image_paths = []
bbox = []
classes = []

for xml_file in tqdm(xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file, path_images)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)

  0%|          | 0/92 [00:00<?, ?it/s]

100%|██████████| 92/92 [00:00<00:00, 1828.25it/s]


In [8]:
def load_image(path):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (640, 640), interpolation=cv2.INTER_CUBIC)
    return image

def load_all_images():
    images = []
    for path in image_paths:
        image = load_image(path)
        images.append(image)
        # print(image.shape)
    return images


images = load_all_images()

In [9]:
image_test_paths = []
bbox_test = []
classes_test = []

for xml_file in tqdm(xml_test_files):
    image_path, boxes, class_ids = parse_annotation(xml_file, path_test_img)
    image_test_paths.append(image_path)
    bbox_test.append(boxes)
    classes_test.append(class_ids)

100%|██████████| 100/100 [00:00<00:00, 1279.80it/s]


In [10]:
def load_image(path):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (640, 640), interpolation=cv2.INTER_CUBIC)
    return image

def load_all_images(paths):
    images = []
    for path in paths:
        image = load_image(path)
        images.append(image)
        # print(image.shape)
    return images


images = load_all_images(image_paths)

test_images = load_all_images(image_test_paths)

In [11]:
images[50].shape

(640, 640, 3)

In [12]:
def drawBoundingBoxes(imageData, imageOutputPath, bboxes, labels, color):
    """Draw bounding boxes on an image.
    imageData: image data in numpy array format
    imageOutputPath: output image file path
    inferenceResults: inference results array off object (l,t,w,h)
    colorMap: Bounding box color candidates, list of RGB tuples.
    """
    i = 0
    for box in bboxes:
        left = int(box[0])
        bottom = int(box[1])
        right = int(box[0]) + int(box[2])
        top = int(box[1]) + int(box[3])
        label = class_mapping[int(labels[i])]
        i+=1
        imgHeight, imgWidth, _ = imageData.shape
        thick = int((imgHeight + imgWidth) // 900)
        print (left, bottom, imgHeight, imgWidth)
        print (label)
        cv2.rectangle(imageData,(left, top), (right, bottom), color, thick)
        cv2.putText(imageData, label, (left, top - 20), 0, 1e-3 * imgHeight, color, thick//3)
    cv2.imwrite(imageOutputPath, imageData)

In [13]:
# print(len(images))
index = 50
drawBoundingBoxes(images[index], '/home/gabriela/projetos/yolov8keras/output/example_Bach1.jpg', bbox[index], classes[index], (0, 0, 255))

285 311 640 640
eosinofilo


In [14]:
images = tf.convert_to_tensor(images, dtype=tf.float32)
# print(images.shape)

I0000 00:00:1737407536.934319    1462 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [15]:
ragged_bboxes = tf.ragged.constant(bbox, dtype=tf.float32)
ragged_classes = tf.ragged.constant(classes, dtype=tf.int64)

labels = {
    "boxes": ragged_bboxes.to_tensor(),
    "classes": ragged_classes.to_tensor(),
}

# print(labels["boxes"].type())
print(len(labels["classes"]))

92


In [16]:
# model = keras_cv.models.YOLOV8Detector(
#     num_classes=len(class_mapping),
#     bounding_box_format="xywh",
#     backbone=keras_cv.models.YOLOV8Backbone.from_preset(
#         "yolo_v8_s_backbone_coco"
#     ),
#     fpn_depth=2
# )

backbone=keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")

model = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xywh",
    backbone=backbone,
    fpn_depth=2,
)

In [17]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

model.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)

dataset = tf.data.Dataset.from_tensor_slices((images, labels["boxes"], labels["classes"]))

# model.compile(
#     classification_loss='binary_crossentropy',
#     box_loss='ciou',
#     optimizer=tf.optimizers.SGD(global_clipnorm=10.0),
#     jit_compile=False,
# )

In [23]:
next(iter(dataset))

(<tf.Tensor: shape=(640, 640, 3), dtype=float32, numpy=
 array([[[10.,  5.,  6.],
         [ 9.,  4.,  5.],
         [ 8.,  3.,  4.],
         ...,
         [ 4.,  0.,  0.],
         [ 7.,  2.,  3.],
         [ 7.,  2.,  3.]],
 
        [[12.,  7.,  8.],
         [ 7.,  2.,  3.],
         [ 5.,  1.,  1.],
         ...,
         [ 8.,  3.,  4.],
         [ 9.,  4.,  5.],
         [ 5.,  0.,  1.]],
 
        [[ 4.,  0.,  0.],
         [ 7.,  2.,  3.],
         [14.,  9., 10.],
         ...,
         [ 7.,  2.,  3.],
         [ 8.,  3.,  4.],
         [13.,  8.,  9.]],
 
        ...,
 
        [[20., 14., 15.],
         [26., 20., 21.],
         [26., 21., 22.],
         ...,
         [ 7.,  2.,  3.],
         [ 6.,  1.,  2.],
         [ 6.,  1.,  2.]],
 
        [[21., 16., 17.],
         [19., 13., 14.],
         [26., 20., 21.],
         ...,
         [10.,  5.,  6.],
         [ 8.,  3.,  4.],
         [ 8.,  3.,  4.]],
 
        [[28., 23., 24.],
         [29., 21., 22.],
         [26

In [22]:
# model.fit(images, labels, batch_size=4, epochs=50)
model.fit(dataset, batch_size=4, epochs=50)

Epoch 1/50


ValueError: too many values to unpack (expected 2)

In [ ]:
y_pred = model.predict(test_images)
print(y_pred)